In [1]:
#export
from k1lib.callbacks import Callback, Callbacks
import k1lib, time, numpy as np, math
from functools import partial
import matplotlib.pyplot as plt

In [2]:
#export
@k1lib.patch(Callback.cls)
class ProgressBar(Callback):
    """Displays the current progress, epoch and batch while running."""
    def startRun(self):
        self.startTime = time.time(); self.step = 0
        self._maxNBatches = 1 # for aesthetic purposes
    def startBatch(self):
        self.learner.elapsedTime = time.time() - self.startTime
        self.step += 1
        self.learner.progress = (self.batch / self.nBatches + self.epoch) / self.epochs
        if self.step % 10 == 0:
            self._maxNBatches = max(self._maxNBatches, self.nBatches)
            a = str(round(100 * self.progress)).rjust(3)
            b = f"{self.epoch}/{self.epochs}".rjust(k1lib.numDigits(self.epochs) * 2 + 1)
            c = f"{self.batch}/{self.nBatches}".rjust(k1lib.numDigits(self._maxNBatches) * 2 + 1)
            d = f"{round(self.elapsedTime, 2)}".rjust(6)
            print(f"\rProgress: {a}%, epoch: {b}, batch: {c}, elapsed: {d}s  ", end="")
@k1lib.patch(Callbacks, docs=ProgressBar)
def withProgressBar(self): return self.append(ProgressBar())

In [3]:
#export
def plotF(losses, _slice): # actual function stored by the sliceable plot
    plt.figure(figsize=(10, 3), dpi=100)
    tR, vR = k1lib.Range.proportionalSlice(len(losses.train), len(losses.valid), _slice)
    try:
        plt.subplot(1, 2, 1); plt.plot(tR.range, losses.train[tR.slice]); plt.title(f"Train loss")
        plt.subplot(1, 2, 2); plt.plot(vR.range, losses.valid[vR.slice]); plt.title(f"Valid loss")
    except: pass
    plt.show()
def commonPlot(obj):
    return k1lib.viz.SliceablePlot(partial(plotF, obj), docs="""\n\nReminder: the actual slice you put in is for the training plot. The valid loss's plot will update automatically to be in the same time frame""")
def nonEmptyList(_list):
    return [0] if _list == [] else _list
@k1lib.patch(Callback.cls)
class Loss(Callback):
    "Records losses after each batch. Display this in your cell for more"
    def __init__(self):
        super().__init__()
        self.train = []; self.valid = [] # all stats all times
        # average stats for each epoch
        self.epoch = k1lib.Object.fromDict({"train": [], "valid": []})\
                        .withRepr("Use...\n" +\
                                 "- `.train` for epoch-averaged training losses\n" +\
                                 "- `.valid` for epoch-averaged validation losses\n" +\
                                 "- `.plot()` to plot the 2 above")
        self.plot = partial(commonPlot, self)
        self.epoch.plot = partial(commonPlot, self.epoch)
        self._trainLosses = []; self._validLosses = []
        self.order = 17
    def endLoss(self):
        if self.model.training:
            self._trainLosses.append(self.loss)
        else: self._validLosses.append(self.loss)
    def endEpoch(self):
        self.train.extend(self._trainLosses); self.epoch.train.append(np.mean(nonEmptyList(self._trainLosses)))
        self.valid.extend(self._validLosses); self.epoch.valid.append(np.mean(nonEmptyList(self._validLosses)))
        self._trainLosses = []; self._validLosses = []
    def __repr__(self):
        return f"""{super()._reprHead}, use...
- cb.train: for all training losses over all epochs and batches (#epochs * #batches)
- cb.valid: for all validation losses over all epochs and batches (#epochs * #batches)
- cb.plot(): to plot the 2 above
- cb.epoch: for average losses of each epochs
{super()._reprCan}"""
@k1lib.patch(Callbacks, docs=Loss)
def withLoss(self): return self.append(Loss())

In [4]:
!../../export.py _callbacks/loss_progress

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.0
Uninstalling k1lib-0.1.0:
  Successfully uninstalled k1lib-0.1.0
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/callbacks.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib/k1lib
copying k1lib/_basics.py -> build/lib/k1lib
copying k1lib/nn.py -> 